In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso
)
%matplotlib inline

/Users/yoshimurahiroyuki/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
train = pd.read_csv('./input/train.csv') #訓練データ
test = pd.read_csv('./input/test.csv') #テストデータ
train = train.drop("Id", axis = 1)
target = train["SalePrice"]
train = train.drop("SalePrice", axis = 1)


In [3]:
train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
5,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal
6,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
7,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal
8,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml
9,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,1,2008,WD,Normal


In [4]:
train.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [5]:
#numerics = train.select_dtypes(include='number').columns.tolist()
#numerics

In [6]:
categoricals = train.select_dtypes(exclude='number').columns.tolist()
categoricals

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [7]:
# カテゴリーのカラムを配列に
categoricals = train.select_dtypes(exclude='number').columns.tolist()

# 配列のカラムを丸ごとダミー化
import category_encoders
ce = category_encoders.OrdinalEncoder(cols=categoricals,handle_unknown='impute')
train[categoricals] = ce.fit_transform(train[categoricals])


# train["SalePrice"]

In [8]:
df_train = pd.read_csv('../input/train.csv')

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,1,65.0,8450,1,1,1,1,1,1,...,0,0,1,1,1,0,2,2008,1,1
1,20,1,80.0,9600,1,1,1,1,1,2,...,0,0,1,1,1,0,5,2007,1,1
2,60,1,68.0,11250,1,1,2,1,1,1,...,0,0,1,1,1,0,9,2008,1,1
3,70,1,60.0,9550,1,1,2,1,1,3,...,0,0,1,1,1,0,2,2006,1,2
4,60,1,84.0,14260,1,1,2,1,1,2,...,0,0,1,1,1,0,12,2008,1,1
5,50,1,85.0,14115,1,1,2,1,1,1,...,0,0,1,2,2,700,10,2009,1,1
6,20,1,75.0,10084,1,1,1,1,1,1,...,0,0,1,1,1,0,8,2007,1,1
7,60,1,NaN,10382,1,1,2,1,1,3,...,0,0,1,1,2,350,11,2009,1,1
8,50,2,51.0,6120,1,1,1,1,1,1,...,0,0,1,1,1,0,4,2008,1,2
9,190,1,50.0,7420,1,1,1,1,1,3,...,0,0,1,1,1,0,1,2008,1,1


In [9]:
params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'subsample': 0.7,
            'subsample_freq': 1,
            'learning_rate': 0.01,
            'num_leaves': 100,
            'min_data_in_leaf':24,
            'feature_fraction': 0.9,
            'lambda_l1': 1,  
            'lambda_l2': 1
            #'device':'gpu'
            }

In [10]:

folds = 5
seed = 666

#kf = StratifiedKFold(n_splits=folds, shuffle=False, random_state=seed)

In [11]:
train_X, val_X , train_y, val_y =  train_test_split(train, target, test_size= 0.2, random_state = 666)


In [12]:
lgb_train = lgb.Dataset(train_X, train_y, categorical_feature=categoricals)
lgb_eval = lgb.Dataset(val_X, val_y, categorical_feature=categoricals)

In [13]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10000,
                valid_sets=(lgb_train, lgb_eval),
                early_stopping_rounds=100,
                verbose_eval = 100)

/Users/yoshimurahiroyuki/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/yoshimurahiroyuki/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 41156.4	valid_1's rmse: 47689.6
[200]	training's rmse: 27725.9	valid_1's rmse: 36693.9
[300]	training's rmse: 22971.7	valid_1's rmse: 32604.1
[400]	training's rmse: 20777	valid_1's rmse: 30910.6
[500]	training's rmse: 19367	valid_1's rmse: 30186.5
[600]	training's rmse: 18201.9	valid_1's rmse: 29702
[700]	training's rmse: 17224.4	valid_1's rmse: 29400.8
[800]	training's rmse: 16319.5	valid_1's rmse: 29088.7
[900]	training's rmse: 15527.3	valid_1's rmse: 28864.9
[1000]	training's rmse: 14801	valid_1's rmse: 28721.9
[1100]	training's rmse: 14181.9	valid_1's rmse: 28640.2
[1200]	training's rmse: 13605.8	valid_1's rmse: 28568.8
[1300]	training's rmse: 13054.5	valid_1's rmse: 28515.2
[1400]	training's rmse: 12537.2	valid_1's rmse: 28474.8
[1500]	training's rmse: 12076.3	valid_1's rmse: 28444
[1600]	training's rmse: 11613.6	valid_1's rmse: 28394.3
[1700]	training's rmse: 11186.1	valid_1's rmse: 28354.3
[180

In [14]:
# fi = pd.DataFrame(  {0: train_X.columns ,  1: gbm.feature_importance()})
# low_fis = fi.sort_values(1).head(20)[0].tolist()

low_fis = ['SaleCondition','LowQualFinSF', 'SaleType', 'Heating', 'GarageQual', 'BsmtFinType2', 'GarageCond',
     'RoofMatl', '3SsnPorch', 'PoolArea', 'Condition2', 'Functional', 'PoolQC', 'Utilities', 'Street', 'MiscVal', 'BldgType', 'MiscFeature', 'RoofStyle', 'Alley']
# print(categoricals)
# for low_fi in low_fis:
#     print (low_fi)
#     features.remove(low_fi)

In [15]:
ce_test = category_encoders.OrdinalEncoder(cols=categoricals,handle_unknown='impute')
test[categoricals] = ce.fit_transform(test[categoricals])

In [16]:

res =gbm.predict(test)



In [17]:
from datetime import datetime
sub = pd.DataFrame()
sub['Id'] = test['Id']
sub['SalePrice'] = res
tmpTime = datetime.now().strftime("%Y%m%d%H%M%S")
filename='submission'+tmpTime+'.csv'
sub.to_csv( filename, index=False)
